In [3]:
!pip install tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     -------------------------- ----------- 41.0/57.7 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\avina\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import os
import time
import random
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# Progress bar
try:
    from tqdm import tqdm
except ImportError:
    raise ImportError("Please install tqdm: pip install tqdm")

# =========================================================
# CONFIG
# =========================================================
#POLYGON_API_KEY = os.getenv("POLYGON_API_KEY", "").strip()
# Or paste temporarily:
POLYGON_API_KEY = "EvEEGfEVBQIiYKRMHGjm9GYZhKWSJ7De"

INPUT_XLSX = r"D:\work\Trade Analysis\Mapping ExERCISE\Stocks Backtesting status.xlsx"
SYMBOL_COL = "Symbol"

START_DATE = "2016-01-01"
END_DATE   = datetime.utcnow().strftime("%Y-%m-%d")

MULTIPLIER = 5
TIMESPAN   = "minute"
ADJUSTED   = "true"
SORT       = "asc"
LIMIT      = 50000

OUT_DIR = Path(r"D:\work\Trade Analysis\Polygon_5min_2016_to_today")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Speed tuning
MAX_WORKERS = 24
MAX_RETRIES = 8

# Global QPS throttle
ENABLE_QPS_LIMIT = True
TARGET_QPS = 6.0

WRITE_EMPTY_MARKER = True

FAIL_LOG = OUT_DIR / "failures.log"
PROG_LOG = OUT_DIR / "progress.log"

HEARTBEAT_SEC = 30  # print a status line every N seconds

BASE_URL = "https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"

# =========================================================
# GLOBAL QPS LIMITER
# =========================================================
_last_req_ts = 0.0
_req_lock = Lock()

def pace_requests():
    global _last_req_ts
    if not ENABLE_QPS_LIMIT:
        return
    min_gap = 1.0 / float(TARGET_QPS)
    with _req_lock:
        now = time.time()
        wait = (_last_req_ts + min_gap) - now
        if wait > 0:
            time.sleep(wait)
        _last_req_ts = time.time()

# =========================================================
# DATE CHUNKS
# =========================================================
def month_chunks(start_date: str, end_date: str):
    s = pd.Timestamp(start_date)
    e = pd.Timestamp(end_date)
    cur = pd.Timestamp(year=s.year, month=s.month, day=1)
    while cur <= e:
        nxt = cur + pd.offsets.MonthBegin(1)
        chunk_start = max(cur, s)
        chunk_end = min(nxt - pd.Timedelta(days=1), e)
        yield chunk_start.strftime("%Y-%m-%d"), chunk_end.strftime("%Y-%m-%d")
        cur = nxt

def month_key(from_date: str) -> str:
    return from_date[:7].replace("-", "_")

def out_path(symbol: str, from_date: str) -> Path:
    return OUT_DIR / symbol / f"{symbol}_5m_{month_key(from_date)}.parquet"

def ensure_symbol_dir(symbol: str):
    (OUT_DIR / symbol).mkdir(parents=True, exist_ok=True)

# =========================================================
# SYMBOL LOADER
# =========================================================
def read_symbols_from_excel(xlsx_path: str):
    x = pd.read_excel(xlsx_path)
    if SYMBOL_COL not in x.columns:
        raise ValueError(f"Could not find '{SYMBOL_COL}'. Available: {list(x.columns)}")

    syms = (
        x[SYMBOL_COL]
        .dropna()
        .astype(str)
        .str.strip()
        .str.upper()
        .tolist()
    )
    seen, out = set(), []
    for s in syms:
        if s and s not in seen:
            out.append(s)
            seen.add(s)
    return out

# =========================================================
# NETWORK
# =========================================================
_session = requests.Session()

def polygon_request(symbol: str, from_d: str, to_d: str):
    url = BASE_URL.format(
        ticker=symbol,
        multiplier=MULTIPLIER,
        timespan=TIMESPAN,
        from_date=from_d,
        to_date=to_d,
    )
    params = {
        "adjusted": ADJUSTED,
        "sort": SORT,
        "limit": LIMIT,
        "apiKey": POLYGON_API_KEY,
    }

    last_err = None
    for attempt in range(MAX_RETRIES):
        try:
            pace_requests()
            r = _session.get(url, params=params, timeout=60)

            if r.status_code == 429:
                time.sleep(min(60, (2 ** attempt)) + random.random())
                continue
            if r.status_code in (500, 502, 503, 504):
                time.sleep(min(30, (2 ** attempt)) + random.random())
                continue

            if r.status_code != 200:
                return None, f"HTTP_{r.status_code}: {r.text[:200]}"

            js = r.json()
            results = js.get("results") or []
            if not results:
                return pd.DataFrame(), "empty"

            df = pd.DataFrame(results).rename(columns={
                "t": "ts_ms",
                "o": "open",
                "h": "high",
                "l": "low",
                "c": "close",
                "v": "volume",
                "vw": "vwap",
                "n": "num_trades",
            })
            df.insert(0, "symbol", symbol)
            df["datetime_utc"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True, errors="coerce")
            return df, "ok"

        except Exception as e:
            last_err = str(e)
            time.sleep(min(30, (2 ** attempt)) + random.random())

    return None, f"error_retry_exhausted: {last_err}"

# =========================================================
# ONE TASK
# =========================================================
def fetch_one(symbol: str, from_d: str, to_d: str):
    fp = out_path(symbol, from_d)
    if fp.exists():
        return (symbol, from_d, "skip", 0)

    df, status = polygon_request(symbol, from_d, to_d)

    if status == "ok":
        df = df.drop_duplicates(subset=["symbol", "ts_ms"]).sort_values(["symbol", "ts_ms"])
        df.to_parquet(fp, index=False)
        return (symbol, from_d, "ok", len(df))

    if status == "empty":
        if WRITE_EMPTY_MARKER:
            pd.DataFrame({"symbol":[symbol], "from":[from_d], "to":[to_d], "note":["empty"]}).to_parquet(fp, index=False)
        return (symbol, from_d, "empty", 0)

    with open(FAIL_LOG, "a", encoding="utf-8") as f:
        f.write(f"{datetime.utcnow().isoformat()}Z | {symbol} | {from_d}->{to_d} | {status}\n")
    return (symbol, from_d, "fail", 0)

# =========================================================
# MAIN
# =========================================================
def main():
    if not POLYGON_API_KEY:
        raise ValueError("Polygon API key missing. Set POLYGON_API_KEY env var or paste it.")

    if not Path(INPUT_XLSX).exists():
        raise FileNotFoundError(f"Excel file not found: {INPUT_XLSX}")

    symbols = read_symbols_from_excel(INPUT_XLSX)
    chunks = list(month_chunks(START_DATE, END_DATE))

    print(f"Loaded symbols: {len(symbols):,}")
    print(f"Months: {len(chunks)} | Range: {START_DATE} -> {END_DATE}")
    print(f"OUT_DIR: {OUT_DIR}")
    print(f"Workers={MAX_WORKERS}, QPS={TARGET_QPS} (limiter={ENABLE_QPS_LIMIT})")

    # Build missing tasks only (resume-safe)
    tasks = []
    for sym in symbols:
        ensure_symbol_dir(sym)
        for from_d, to_d in chunks:
            if not out_path(sym, from_d).exists():
                tasks.append((sym, from_d, to_d))

    total = len(tasks)
    print(f"Tasks to fetch (missing only): {total:,}")
    if total == 0:
        print("Nothing to do.")
        return

    ok = empty = skip = fail = 0
    rows_saved = 0
    t0 = time.time()
    last_heartbeat = t0

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex, tqdm(total=total, desc="Polygon 5m download") as pbar:
        futs = [ex.submit(fetch_one, sym, fd, td) for (sym, fd, td) in tasks]

        for i, fut in enumerate(as_completed(futs), start=1):
            sym, fd, st, nrows = fut.result()

            if st == "ok":
                ok += 1
                rows_saved += nrows
            elif st == "empty":
                empty += 1
            elif st == "skip":
                skip += 1
            else:
                fail += 1

            # update progress bar
            pbar.update(1)
            pbar.set_postfix({
                "ok": ok,
                "empty": empty,
                "fail": fail,
                "rows": f"{rows_saved/1e6:.1f}M"
            })

            # heartbeat print
            now = time.time()
            if now - last_heartbeat >= HEARTBEAT_SEC:
                elapsed = now - t0
                rate = i / elapsed if elapsed > 0 else 0.0
                remaining = total - i
                eta_sec = remaining / rate if rate > 0 else float("inf")
                msg = (f"[{datetime.now().strftime('%H:%M:%S')}] {i:,}/{total:,} | "
                       f"ok={ok:,} empty={empty:,} fail={fail:,} | "
                       f"{rate:.2f} tasks/sec | ETA ~ {eta_sec/60:.1f} min")
                print(msg)
                with open(PROG_LOG, "a", encoding="utf-8") as f:
                    f.write(msg + "\n")
                last_heartbeat = now

    print("\nDONE")
    print(f"ok={ok:,} empty={empty:,} fail={fail:,} rows_saved={rows_saved:,}")
    print("Failure log:", FAIL_LOG)
    print("Progress log:", PROG_LOG)

if __name__ == "__main__":
    main()


Loaded symbols: 1,789
Months: 121 | Range: 2016-01-01 -> 2026-01-23
OUT_DIR: D:\work\Trade Analysis\Polygon_5min_2016_to_today
Workers=24, QPS=6.0 (limiter=True)
Tasks to fetch (missing only): 178,214


Polygon 5m download:   0%|          | 176/178214 [00:30<7:27:32,  6.63it/s, ok=161, empty=15, fail=0, rows=0.2M] 

[16:47:47] 174/178,214 | ok=159 empty=15 fail=0 | 5.78 tasks/sec | ETA ~ 513.1 min


Polygon 5m download:   0%|          | 354/178214 [01:00<7:26:56,  6.63it/s, ok=339, empty=15, fail=0, rows=0.5M] 

[16:48:17] 352/178,214 | ok=337 empty=15 fail=0 | 5.85 tasks/sec | ETA ~ 507.0 min


...

[01:04:48] 177,976/178,214 | ok=171,102 empty=6,874 fail=0 | 5.96 tasks/sec | ETA ~ 0.7 min


Polygon 5m download: 100%|█████████▉| 178155/178214 [8:18:00<00:08,  6.96it/s, ok=171266, empty=6889, fail=0, rows=226.7M]

[01:05:18] 178,155/178,214 | ok=171,266 empty=6,889 fail=0 | 5.96 tasks/sec | ETA ~ 0.2 min


Polygon 5m download: 100%|██████████| 178214/178214 [8:18:10<00:00,  5.96it/s, ok=171325, empty=6889, fail=0, rows=226.8M]



DONE
ok=171,325 empty=6,889 fail=0 rows_saved=226,777,842
Failure log: D:\work\Trade Analysis\Polygon_5min_2016_to_today\failures.log
Progress log: D:\work\Trade Analysis\Polygon_5min_2016_to_today\progress.log
